In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

# === Pydantic 모델 정의 ===
class StudentIntro(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이 (숫자)")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 목록 (문자열 리스트)")
    goal: str = Field(description="장기적인 목표 또는 꿈")

# 출력 파서
parser = PydanticOutputParser(pydantic_object=StudentIntro)

# === 프롬프트 템플릿 ===
template = """
다음 학생의 자기소개를 읽고, 이름, 나이, 전공, 취미, 목표를 각각 추출해 주세요.
각 항목은 다음 형식에 맞게 출력해 주세요:

{format_instructions}

자기소개:
{text}
"""

# 프롬프트 구성
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# === 모델 초기화 ===
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# === 테스트 입력 ===
intro_text = (
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. "
    "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. "
    "앞으로 훌륭한 개발자가 되는 것이 목표입니다."
)

# === 체인 구성 및 실행 ===
chain = prompt | model | parser
output = chain.invoke({"text": intro_text})

# === 출력 결과 ===
print("\n=== 자기소개 정보 추출 결과 ===")
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {', '.join(output.hobbies)}")
print(f"목표: {output.goal}")



=== 자기소개 정보 추출 결과 ===
이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
